# Generate shape coefficients

The following steps are performed to obtain all the artifacts for shape coefficient use case:

1. Read parameters file
2. Read Mesh description
   * Its path can be specified
   * Or it can be obtained from the simulation configuration file
3. Read data for pressure coefficient

In [1]:
from cfdmod.use_cases.pressure.shape.Ce_config import CeConfig
import pathlib

post_proc_cfg_path = pathlib.Path("./fixtures/tests/pressure/Ce_params.yaml") 
post_proc_cfg = CeConfig.from_file(post_proc_cfg_path)

post_proc_cfg

{'pattern_1': CeConfig(zoning=ZoningConfig(global_zoning=ZoningModel(x_intervals=[0.0, 100.0], y_intervals=[0.0, 50.0], z_intervals=[0.0, 20.0]), no_zoning=['surface1'], exclude=['surface1'], exceptions={'zoning1': ExceptionZoningModel(x_intervals=[0.0, 100.0], y_intervals=[0.0, 50.0], z_intervals=[0.0, 20.0], surfaces=['surface1'])}), statistics=['min', 'max', 'std', 'avg', 'skewness', 'kurtosis']),
 'pattern_2': CeConfig(zoning=ZoningConfig(global_zoning=ZoningModel(x_intervals=[0.0, 100.0], y_intervals=[0.0, 50.0], z_intervals=[0.0, 20.0]), no_zoning=None, exclude=None, exceptions=None), statistics=['min', 'max', 'std', 'avg', 'skewness', 'kurtosis'])}